In [1]:
import os
from typing import Optional
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import torch
import torchvision
from torch import nn

from PIL import Image
from torch.utils.data import Dataset, DataLoader

from torchmetrics import Accuracy, F1Score

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import WandbLogger


from torchvision.transforms import RandomHorizontalFlip, RandomRotation, ColorJitter, GaussianBlur, RandomGrayscale, \
    Compose, Resize, RandomApply, ToTensor, Normalize
pl.seed_everything(34)

Global seed set to 34


34

In [2]:
data_path = '../data/wild_images'

In [3]:
full_df = pd.read_csv(os.path.join(data_path, 'full_wild_images_new.csv'))

In [4]:
full_df.head()

,file_path,subfolder,age,sex,race,age_range,image_size,image_width,image_height,mode
0,34_0_1_20170112213542279.jpg,part2,34,0,1,30_to_40,"(646, 643)",646,643,RGB
1,32_1_0_20170109141308065.jpg,part1,32,1,0,30_to_40,"(638, 319)",638,319,RGB
2,35_1_0_20170116211804087.jpg,part2,35,1,0,30_to_40,"(930, 875)",930,875,RGB
3,21_0_0_20170104020830476.jpg,part1,21,0,0,20_to_30,"(900, 601)",900,601,RGB
4,27_1_2_20170104020523491.jpg,part1,27,1,2,20_to_30,"(518, 601)",518,601,RGB


In [5]:
le = LabelEncoder()
le.fit(full_df.age_range)

LabelEncoder()

In [6]:
le.classes_

array(['0_to_10', '10_to_20', '20_to_30', '30_to_40', '40_to_50',
       '50_to_60', '60_to_70', '70_to_80', 'above_80'], dtype=object)

In [7]:
ages = le.transform(full_df['age_range'])

In [8]:
full_df['target_age'] = ages
full_df.head()

,file_path,subfolder,age,sex,race,age_range,image_size,image_width,image_height,mode,target_age
0,34_0_1_20170112213542279.jpg,part2,34,0,1,30_to_40,"(646, 643)",646,643,RGB,3
1,32_1_0_20170109141308065.jpg,part1,32,1,0,30_to_40,"(638, 319)",638,319,RGB,3
2,35_1_0_20170116211804087.jpg,part2,35,1,0,30_to_40,"(930, 875)",930,875,RGB,3
3,21_0_0_20170104020830476.jpg,part1,21,0,0,20_to_30,"(900, 601)",900,601,RGB,2
4,27_1_2_20170104020523491.jpg,part1,27,1,2,20_to_30,"(518, 601)",518,601,RGB,2


In [9]:
class AgeDataset(Dataset):
    def __init__(self, data, transforms, path='../data/wild_images'):
        self.data = data
        self.transforms = transforms
        self.path = path

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, index):
        row = self.data.iloc[index]
        image_id = row['file_path']
        target_age = row['target_age']
        target_sex = row['sex']
        target_race = row['race']
        subfolder = row['subfolder']
        the_image = Image.open(os.path.join(self.path, subfolder, image_id))
        transformed_image = self.transforms(the_image)
        return transformed_image, target_age, target_sex, target_race

In [10]:
transforms = Compose([Resize((256, 256)), ToTensor()])

In [11]:
the_data = AgeDataset(full_df, transforms, data_path)

In [10]:
class AgePredictionData(pl.LightningDataModule):
    def __init__(self, full_data, train_batch=64, path='../data/wild_images'):
        super().__init__()
        self.path = path
        self.full_data = full_data
        self.augmentations = [
            RandomHorizontalFlip(0.5),
            RandomRotation(30),
            ColorJitter(brightness=.5, hue=.3),
            GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
            RandomGrayscale()
        ]
        self.train_transforms = Compose([
            Resize((256, 256)),
            RandomApply(self.augmentations, p=0.6),
            ToTensor(),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        self.transforms = Compose([Resize((256, 256)), ToTensor(),
                                   Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
        self.train_batch = train_batch
        self.val_batch = train_batch * 4

    def setup(self, stage: Optional[str] = None):
        if stage == 'fit' or stage is None:
            train_data, self.test_data = train_test_split(self.full_data, test_size=0.05, 
                                                     stratify=self.full_data['target_age'])
            self.train, self.val = train_test_split(train_data, test_size=0.15, stratify=train_data['target_age'])
            self.train_data = AgeDataset(self.train, self.train_transforms, self.path)
            self.val_data = AgeDataset(self.val, self.transforms, self.path)

        if stage == 'predict':
            self.pred_data = AgeDataset(self.full_data, self.transforms, self.path)

    def train_dataloader(self):
        return DataLoader(self.train_data, batch_size=self.train_batch, shuffle=True, pin_memory=True,
                          num_workers=16)

    def val_dataloader(self):
        return DataLoader(self.val_data, batch_size=self.val_batch, shuffle=False, pin_memory=True,
                          num_workers=16)

    def predict_dataloader(self):
        return DataLoader(self.pred_data, batch_size=self.val_batch, shuffle=False, pin_memory=True, num_workers=16)

In [11]:
module = AgePredictionData(full_df, 64, data_path)

In [14]:
module.setup()

In [18]:
len(module.train_data), len(module.val_data), len(module.test_data)

(19449, 3433, 1205)

In [19]:
sample_batch = next(iter(module.train_dataloader()))

In [29]:
sample_batch[0].shape

torch.Size([64, 3, 256, 256])

In [12]:
class AgePredictResnet(nn.Module):
    def __init__(self, age_num_targets=9, gender_num=2, race_num=5):
        super().__init__()

        self.model = torchvision.models.resnet101(pretrained=True)
        self.model.fc = nn.Linear(2048, 512)
        self.age_linear1 = nn.Linear(512, 256)
        self.age_linear2 = nn.Linear(256, 128)
        self.age_out = nn.Linear(128, age_num_targets)
        self.gender_linear1 = nn.Linear(512, 256)
        self.gender_linear2 = nn.Linear(256, 128)
        self.gender_out = nn.Linear(128, gender_num)
        self.race_linear1 = nn.Linear(512, 256)
        self.race_linear2 = nn.Linear(256, 128)
        self.race_out = nn.Linear(128, race_num)
        self.activation = nn.ReLU()
        self.dropout = nn.Dropout(0.4)

    def forward(self, x):
        out = self.activation(self.model(x))
        age_out = self.activation(self.dropout((self.age_linear1(out))))
        age_out = self.activation(self.dropout(self.age_linear2(age_out)))
        age_out = self.age_out(age_out)
        
        gender_out = self.activation(self.dropout((self.gender_linear1(out))))
        gender_out = self.activation(self.dropout(self.gender_linear2(gender_out)))
        gender_out = self.gender_out(gender_out)
        
        race_out = self.activation(self.dropout((self.race_linear1(out))))
        race_out = self.activation(self.dropout(self.race_linear2(race_out)))
        race_out = self.race_out(race_out)
        return age_out, gender_out, race_out

In [40]:
model = AgePredictResnet()

In [48]:
outputs = model(sample_batch[0])

In [54]:
outputs[0].shape, outputs[1].shape, outputs[2].shape

(torch.Size([64, 9]), torch.Size([64, 2]), torch.Size([64, 5]))

In [14]:
class AgePrediction(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = AgePredictResnet()
        self.criterion = nn.CrossEntropyLoss()
        self.acc = Accuracy()
        self.f1 = F1Score()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, image, transforms):
        print("inside forward")
        image = transforms(image)
        image = torch.unsqueeze(image, 0)
        print('the image dimensions are :', image.shape)
        self.model.eval()
        logits = self.model(image)
        print("the logits are:", logits)
        pred_probabilities = self.softmax(logits)
        print("The probabilities : ", pred_probabilities)
        prediction = torch.argmax(pred_probabilities, dim=1)
        return prediction

    def training_step(self, input_batch, batch_idx):
        image_tensors = input_batch[0]
        age_targets = input_batch[1]
        sex_targets = input_batch[2]
        race_targets = input_batch[3]

        logits = self.model(image_tensors)
        age_loss = self.criterion(logits[0], age_targets)
        sex_loss = self.criterion(logits[1], sex_targets)
        race_loss = self.criterion(logits[2], race_targets)
        
        total_loss = age_loss + sex_loss + race_loss

        age_predict = torch.argmax(logits[0], dim=1)
        sex_predict = torch.argmax(logits[1], dim=1)
        race_predict = torch.argmax(logits[2], dim=1)
        age_acc = self.acc(age_predict, age_targets)
        sex_acc = self.acc(sex_predict, sex_targets)
        race_acc = self.acc(race_predict, race_targets)
        
        total_acc = (age_acc + sex_acc + race_acc)/3

        self.log("age-loss", age_loss, on_step=True, on_epoch=True, prog_bar=True)
        self.log("sex-loss", sex_loss, on_step=True, on_epoch=True, prog_bar=True)
        self.log("race-loss", race_loss, on_step=True, on_epoch=True, prog_bar=True)
        self.log("total-loss", total_loss, on_step=True, on_epoch=True, prog_bar=True)
        self.log('age-acc', age_acc, on_step=False, on_epoch=True, prog_bar=True)
        self.log('sex-acc', sex_acc, on_step=False, on_epoch=True, prog_bar=True)
        self.log('race-acc', race_acc, on_step=False, on_epoch=True, prog_bar=True)
        self.log('total-train-acc', total_acc, on_step=False, on_epoch=True, prog_bar=True)

        return total_loss

    def validation_step(self, input_batch, batch_idx):
        image_tensors = input_batch[0]
        age_targets = input_batch[1]
        sex_targets = input_batch[2]
        race_targets = input_batch[3]

        logits = self.model(image_tensors)

        age_loss = self.criterion(logits[0], age_targets)
        sex_loss = self.criterion(logits[1], sex_targets)
        race_loss = self.criterion(logits[2], race_targets)
        
        total_loss = age_loss + sex_loss + race_loss
        
        age_predict = torch.argmax(logits[0], dim=1)
        sex_predict = torch.argmax(logits[1], dim=1)
        race_predict = torch.argmax(logits[2], dim=1)
        age_acc = self.acc(age_predict, age_targets)
        sex_acc = self.acc(sex_predict, sex_targets)
        race_acc = self.acc(race_predict, race_targets)
        
        total_acc = (age_acc + sex_acc + race_acc)/3
        

        self.log("val-total-loss", total_loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log('val-age-acc', age_acc, on_step=False, on_epoch=True, prog_bar=True)
        self.log('val-sex-acc', sex_acc, on_step=False, on_epoch=True, prog_bar=True)
        self.log('val-race-acc', race_acc, on_step=False, on_epoch=True, prog_bar=True)
        self.log('val-total-acc', total_acc, on_step=False, on_epoch=True, prog_bar=True)

    def predict_step(self, input_batch, batch_idx):
        print("inside predict_step")
        image_tensors = input_batch[0]
        targets = input_batch[1]

        logits = self.model(image_tensors)

        predictions = torch.argmax(logits, dim=1)
        return predictions.cpu().detach().numpy()

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=0.00002)

In [15]:
train_module = AgePrediction()

In [16]:
data_module = AgePredictionData(full_df, 64, data_path)

In [17]:
checkpoint_callback = ModelCheckpoint(filename='{epoch}-{val-total-acc:.3f}', save_top_k=2, monitor='val-total-acc'
                                          , mode='max')

In [18]:
early_stopping = EarlyStopping(monitor="val-total-acc", patience=10, verbose=False, mode="max")

In [19]:
wandb_logger = WandbLogger(project="UTK_Age_Prediction", save_dir='../lightning_logs',
                               name="resnet101_first_model")

wandb: Currently logged in as: nikhilsalodkar. Use `wandb login --relogin` to force relogin


In [20]:
trainer = pl.Trainer(accelerator='gpu', fast_dev_run=False, max_epochs=100,
                         callbacks=[checkpoint_callback, early_stopping], logger=wandb_logger, precision=16)

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [21]:
trainer.fit(train_module, data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | AgePredictResnet | 44.0 M
1 | criterion | CrossEntropyLoss | 0     
2 | acc       | Accuracy         | 0     
3 | f1        | F1Score          | 0     
4 | softmax   | Softmax          | 0     
-----------------------------------------------
44.0 M    Trainable params
0         Non-trainable params
44.0 M    Total params
88.088    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]